In [ ]:
!pip install pyngrok
!pip install flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=462d1d60ec281e6bef8f77f7ccb9cddf87e703174996d786597ed9ac421876f8
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [ ]:
!pip install transformers[sentencepiece] datasets spacy scipy networkx numpy sent2vec pyngrok pandas nltk

In [ ]:
# import library yang dibutuhkan
import torch
import spacy
import networkx as nx
import numpy as np

from scipy import spatial
from sent2vec.vectorizer import Vectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import TFPegasusForConditionalGeneration, PegasusTokenizerFast
from transformers import BertTokenizer, BertModel

import re
import unicodedata

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**PERINGKASAN BAHASA INGGRIS**

In [ ]:
# 1. FUNGSI PRE-PROCESSING
def preprocess_en(text):
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(text)

  # Segmentasi (pisahkan text per kalimat dan masukkan kedalam list)
  sentences = [sent.text.strip() for sent in doc.sents]

  # Membersihkan setiap kalimat
  filtered_sentences = []
  for i in range(len(sentences)):
    doc = nlp(sentences[i])

    # inisial variabel untuk menampung kata yang bersih
    filtered_tokens = []

    # tokenisasi
    for token in doc:
    # menghilangkan stopword dan karakter yang tidak dibutuhkan
      if token.is_stop or token.is_punct:
        continue
      # lemmatisasi
      filtered_tokens.append(token.lemma_)

    # how to convert list to string
    result = " ".join(filtered_tokens)

    filtered_sentences.append(result)
  return filtered_sentences

In [ ]:
# Load the tokenizer and model
tokenizer_bert_en = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert_en = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
def vector_en(list_sentences):

  # Initialize an empty array to store sentence embeddings
  sentence_embeddings = []

  # Iterate through sentences and get embeddings
  for text in list_sentences:
      input_ids = tokenizer_bert_en.encode(text, add_special_tokens=True, padding=True, truncation=True)
      input_ids = torch.tensor(input_ids).unsqueeze(0)

      with torch.no_grad():
          outputs = model_bert_en(input_ids)
          cls_embedding = outputs.last_hidden_state[:, 0, :]
          embedding = cls_embedding[0]

      sentence_embeddings.append(embedding.numpy())

  return sentence_embeddings

In [ ]:
# 2. FUNGSI PERINGKASAN EKSTRAKTIF
def extractive_sum_en(filtered_sentences, sentences):
  # mengubah kalimat menjadi vektor
  vectors = vector_en(filtered_sentences)

  # menghitung similarity matrix (matriks kemiripan antar kalimat)
  similarity_matrix = []
  for i in range(len(vectors)):
    row = []
    for j in range(len(vectors)):
      row.append(spatial.distance.cosine(vectors[i], vectors[j]))
    similarity_matrix.append(row)

  # konversi matrix menjadi graph
  graph = nx.from_numpy_array(np.array(similarity_matrix))

  # melakukan perangkingan
  scores = nx.pagerank(graph)

  # mengambil top rank (kalimat dengan score tertinggi)
  sentences_size = len(sentences)
  num_sentences = round((sentences_size + 1) / 2)

  top_sentence_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:num_sentences]
  summary = [sentences[i] for i in top_sentence_indices]
  summary = " ".join(summary)

  # hasil peringkasan ekstraktif
  return summary


In [ ]:
# 3. FUNGSI PERINGKASAN ABSTRAKTIF

model_name1 = "google/pegasus-cnn_dailymail"
model1 = PegasusForConditionalGeneration.from_pretrained(model_name1)
tokenizer1 = PegasusTokenizer.from_pretrained(model_name1)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def abstractive_sum_en(text):

  inputs = tokenizer1.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)

  summary_ids = model1.generate(inputs, max_length=250, min_length=50, length_penalty=2.0, num_beams=6, early_stopping=True)

  summary = tokenizer1.decode(summary_ids[0], skip_special_tokens=True)

  return summary

In [ ]:
# MERINGKAS TEKS BAHASA INGGRIS
def summy_en(text):
  document = text
  # load spacy
  nlp = spacy.load("en_core_web_sm")

  # memasukkan kalimat kedalam list (untuk memudahkan print output)
  doc = nlp(document)
  sentences = [sent.text.strip() for sent in doc.sents]

  # melakukan pre-processing
  # contoh hasil penggunaan fungsi preprocess
  result = preprocess_en(document)

  # melakukan peringkasan ekstraktif
  summary1 = extractive_sum_en(result, sentences)

  # melakukan peringkasan abstraktif
  summary = abstractive_sum_en(summary1)
  summary = summary.replace("<pad>", "")
  summary = summary.replace("</s>", "")
  summary = summary.replace(".<n>", ".\n")

  return summary


In [ ]:
text = """

Python is a high-level, interpreted programming language that is widely used for web development, data analysis, artificial intelligence, and many other applications. Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages in the world, thanks to its simplicity, versatility, and wide range of libraries and frameworks.

Python is known for its elegant syntax, which emphasizes readability and reduces the cost of program maintenance. Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. This makes Python code more concise and easier to read, especially for beginners who are just learning how to program.

In addition to its simplicity and readability, Python is also praised for its powerful and flexible features. Python supports object-oriented programming, functional programming, and procedural programming paradigms, making it a versatile tool for different kinds of applications. Python also has a large standard library and a vast ecosystem of third-party libraries and packages, which makes it easy to find and use pre-built modules for different purposes.

Whether you are a beginner or an experienced programmer, learning Python can open up many opportunities for you. With its growing popularity and wide range of applications, Python is an essential skill for many industries and fields. So why not start learning Python today and see where it takes you?

"""

In [ ]:
result_en = summy_en(text)

In [ ]:
print(result_en)

Python supports object-oriented programming, functional programming, and procedural programming paradigms.
Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation.
 Python is an essential skill for many industries and fields.


**PERINGKASAN BAHASA INDONESIA**

In [ ]:
#1. PRE-PROCESSING
def preprocess_id(text):

  # Segmentasi (pisahkan text per kalimat dan masukkan kedalam list)
  sentences = sent_tokenize(text)

  # Membersihkan setiap kalimat
  filtered_sentences = []
  for i in range(len(sentences)):

    text = sentences[i]

    # Tokenisasi (memecah kalimat menjadi kata)
    words = word_tokenize(text.lower())

    # Membersihkan tanda baca
    words = [word for word in words if word.isalnum()]

    # Menghapus stopwords
    stop_words = set(stopwords.words("indonesian"))
    words = [word for word in words if word not in stop_words]

    # Melakukan lemmatisasi
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Menggabungkan kembali kata yang telah dibersihkan menjadi kalimat
    result = " ".join(lemmatized_words)

    filtered_sentences.append(result)
  return filtered_sentences

In [ ]:
# Load the tokenizer and model
tokenizer_bert_id = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model_bert_id = BertModel.from_pretrained('bert-base-multilingual-cased')

In [ ]:
def vector_id(list_sentences):

  # Initialize an empty array to store sentence embeddings
  sentence_embeddings = []

  # Iterate through sentences and get embeddings
  for text in list_sentences:
      input_ids = tokenizer_bert_id.encode(text, add_special_tokens=True, padding=True, truncation=True)
      input_ids = torch.tensor(input_ids).unsqueeze(0)

      with torch.no_grad():
          outputs = model_bert_id(input_ids)
          cls_embedding = outputs.last_hidden_state[:, 0, :]
          embedding = cls_embedding[0]

      sentence_embeddings.append(embedding.numpy())

  return sentence_embeddings

In [ ]:
# 2. PERINGKASAN EKSTRAKTIF

def extractive_sum_id(filtered_sentences, sentences):
  # mengubah kalimat menjadi vektor
  vectors = vector_id(filtered_sentences)

  # menghitung similarity matrix (matriks kemiripan antar kalimat)
  similarity_matrix = []
  for i in range(len(vectors)):
    row = []
    for j in range(len(vectors)):
      row.append(spatial.distance.cosine(vectors[i], vectors[j]))
    similarity_matrix.append(row)

  # konversi matrix menjadi graph
  graph = nx.from_numpy_array(np.array(similarity_matrix))

  # melakukan perangkingan
  scores = nx.pagerank(graph)

  # mengambil top rank (kalimat dengan score tertinggi)
  sentences_size = len(sentences)
  num_sentences = round((sentences_size + 1) / 2)

  top_sentence_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:num_sentences]
  summary = [sentences[i] for i in top_sentence_indices]

  # hasil peringkasan ekstraktif

  return " ".join(summary)

In [ ]:
# 3. PERINGKASAN ABSTRAKTIF

# membersihkan teks
def text_cleaning(input_string):
    lowercase = input_string.lower()
    remove_link = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)', '', lowercase).replace("&amp;","&")
    remove_bullet = "\n".join([T for T in remove_link.split('\n') if '•' not in T and "baca juga:" not in T])
    remove_accented = unicodedata.normalize('NFKD', remove_bullet).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    remove_parentheses = re.sub("([\(\|]).*?([\)\|])", "\g<1>\g<2>", remove_accented)
    remove_punc = re.sub(r"[^\w\d.\s]+",' ', remove_parentheses)
    remove_num_dot = re.sub(r"(?<=\d)\.|\.(?=\d)|(?<=#)\.","", remove_punc)
    remove_extra_whitespace =  re.sub(r'^\s*|\s\s*', ' ', remove_num_dot).strip()
    return ".".join([s for s in remove_extra_whitespace.strip().split('.') if len(s.strip())>10]).replace("_","")

In [ ]:
model_name2 = "thonyyy/pegasus_indonesian_base-finetune"
model2 = TFPegasusForConditionalGeneration.from_pretrained(model_name2)
tokenizer2 = PegasusTokenizerFast.from_pretrained(model_name2)

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

All the layers of TFPegasusForConditionalGeneration were initialized from the model checkpoint at thonyyy/pegasus_indonesian_base-finetune.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFPegasusForConditionalGeneration for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def abstractive_sum_id(text):
  clean_text = text_cleaning(text)
  inputs = tokenizer2(clean_text, return_tensors = 'tf')
  summary_ids = model2.generate(**inputs, max_new_tokens = 256, length_penalty=2.0, num_beams=4, early_stopping=True)
  summary = tokenizer2.batch_decode(summary_ids, skip_special_tokens=True)

  return summary[0]

In [ ]:
# MERINGKAS TEKS BAHASA
def summy_id(text):

  # memasukkan kalimat kedalam list (untuk memudahkan print output)
  sentences = sent_tokenize(text)

  # 1. PRE-PROCESSING
  result = preprocess_id(text)

  # 2. PERINGKASAN EKSTRAKTIF
  summary1 = extractive_sum_id(result, sentences)

  # 3. PERINGKASAN ABSTRAKTIF
  summary = abstractive_sum_id(summary1)
  summary = summary.replace("<pad>", "")
  summary = summary.replace("</s>", "")
  summary = summary.replace(".<n>", ".\n")

  return summary

In [ ]:
text = """

Sampah selalu kita temukan mengotori lingkungan di sekitar kita. Maka wajar karena hal itu seringkali sampah menjadi masalah lingkungan yang serius harus ditangani.

Sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. Walaupun sampah jelas-jelas membuat lingkungan tidak nyaman tetapi anehnya kesadaran kita terhadap lingkungan masih jauh dari cukup.

Masih banyak di antara kita yang tidak memperhatikan membuang sampah pada tempatnya. Mereka baru menyadari pentingnya membuang sampah secara disiplin, ketika mulai banyak rusaknya lingkungan diakibatkan oleh sampah yang menumpuk.

Pada akhirnya kondisi ini telah membuat banyak orang menjadi sadar bahwa mengelola sampah dengan bijak sangatlah penting untuk menjamin rasa nyaman lingkungan juga memperhatikan kesehatan.

Gerakan untuk membuang sampah secara disiplin pada tempat sampah semakin ramai digaungkan. Tempat-tempat sampah semakin berlimpah disediakan di depan rumah-rumah. Kondisi ini sungguh sangat menggembirakan.

Ibu-ibu pun mulai mengajarkan kepada anak-anak mereka sejak usia dini, betapa pentingnya membuang sampah dengan disiplin pada tempat yang sudah disediakan.

Karena hal itu akan menjamin kebersihan lingkungan semakin sehat bagi kehidupan mereka sehari-hari. Disiplin yang ditanamkan sejak usia dini akan menumbuhkan pribadi dengan karakter unggul.

"""

In [ ]:
result_id = summy_id(text)

In [ ]:
print(result_id)

sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. disiplin yang ditanamkan sejak usia dini akan menumbuhkan pribadi dengan karakter unggul


In [ ]:
# validasi data inputan
def count_word(text):
   return len(text.split())

RUNNING ON FLASK

In [ ]:
from flask import Flask
from pyngrok import ngrok

In [ ]:
port_no = 5000

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from flask import url_for, request
from flask.templating import render_template
template_folder = "/content/gdrive/MyDrive/NLP/f_zero/templates"
static_folder = "/content/gdrive/MyDrive/NLP/f_zero/static"
app = Flask(__name__, template_folder=template_folder, static_folder=static_folder)
ngrok.set_auth_token("2TI0JTA7MxqbjwsmutMmvwUshCV_4b51PZvttuYQYFpur9FJp")
public_url = ngrok.connect(port_no).public_url

@app.route("/index", methods=["POST", "GET"])
def index():
  output = ""
  v_textarea = ""
  language = ""
  n_words = 0
  if request.method == 'POST':
     input_text = request.form['Textarea']
     language = request.form['select1']
     # validasi jumlah kata minimal 100
     n_words = count_word(input_text)
     if n_words <= 100:
        output = "*Input sentences are too concise"
     elif n_words > 500:
        output = "*Input more than 500 words"
     else:
        if language == "english" :
          output = summy_en(input_text)
        elif language == "indonesia" :
          output = summy_id(input_text)
        else:
          output = "bahasa tidak terdeteksi"
     v_textarea = request.form['Textarea']
  return render_template("index.html", output=output, v_textarea = v_textarea)

print(f"To access the global link please click {public_url}")


app.run(port=port_no)

To access the global link please click https://48cb-35-196-250-47.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 05:04:59] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 05:05:00] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 05:05:05] "GET /index HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 05:05:06] "GET /static/index.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 05:05:06] "GET /static/people.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 05:06:33] "POST /index HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 05:06:34] "GET /static/people.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 05:06:34] "GET /static/index.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 05:08:11] "POST /index HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/